In [1]:
import glob
import pandas as pd
import numpy as np
import os
import csv
import sys

Import and concatenate the csv files containing the persons and households of the area of study

In [2]:
## Import multiple csv files into pandas and concatenate into one DataFrame
# get data file names from the persons
path_persons = r'C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\SPENSER\Data\2019\2019_persons' 
all_persons_files = glob.iglob(os.path.join(path_persons, "*.csv"))

# get data file names from the households
path_households = r'C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\SPENSER\Data\2019\2019_households'
all_households_files = glob.iglob(os.path.join(path_households, "*.csv"))


In [3]:
#concatenate all persons files in one dataframe
persons_temp = []

for filename in all_persons_files:
    df_persons = pd.read_csv(filename, index_col=None, header=0)
    persons_temp.append(df_persons)

df_persons_NE = pd.concat(persons_temp, axis=0, ignore_index=True)


In [4]:
#concatenate all households files in one dataframe
households_temp = []

for filename in all_households_files:
    df_households = pd.read_csv(filename, index_col=None, header=0)
    households_temp.append(df_households)

df_households_NE = pd.concat(households_temp, axis=0, ignore_index=True)

Read and save in a list with the values of MSOA and OA areas 

This file was create in PostGIS with the following SQL query:

Create table boundaries.MSOA_OA_2011_ne as

select a.msoa11cd as MSOA, b.oa11cd as OA

from boundaries.msoa_2011_ne a, boundaries.oa_2011_ne b

where ST_Within(b.geom, a.geom)


(MSOA and OA areas were downloaded from https://geoportal.statistics.gov.uk/)

In [5]:
#Read CSV file containing the MSOA and OA values only from the North East of England
MSOA_OA_2011_NE_dir = r'C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement' # use your path
MSOA_OA_2011_NE_files = os.path.join(MSOA_OA_2011_NE_dir, "MSOA_OA_2011_NE.csv")
df_MSOA_OA_2011_NE_list = pd.read_csv(MSOA_OA_2011_NE_files, index_col=None, header=0)

In [6]:
# Check the total number of MSOA levels are in the dataframe
# counting unique values of MSOA levels in the df_persons_NE
n = len(pd.unique(df_persons_NE['Area']))
  
print("No.of.unique values :", 
      n)

No.of.unique values : 340


In [7]:
# Check the total number of OA areas are in the dataframe
# counting unique values of OA areas in the df_households_NE
m = len(pd.unique(df_households_NE['Area']))
  
print("No.of.unique values :", 
      m)

No.of.unique values : 8802


Rename and convert some column names and values

In [9]:
#Rename the column names of 'df_persons_NE'
df_persons_NE = df_persons_NE.rename({'Area': 'Area_MSOA', 'DC1117EW_C_SEX': 'Sex', 'DC1117EW_C_AGE': 'Age', 'DC2101EW_C_ETHPUK11': 'Ethnic'}, axis=1)

In [10]:
#Rename the column names of 'df_households_NE'
df_households_NE = df_households_NE.rename({'Area': 'Area_OA'}, axis=1)

In [11]:
# convert column "HID" to int
df_households_NE = df_households_NE.astype({"HID": int})

Generate a unique PRIMARY KEY for df_persons_NE and df_households_NE

In [12]:
#Create a new column in 'df_persons_NE' for the unique ID --> PID_Area_MSOA
df_persons_NE["PID_AreaMSOA"] = df_persons_NE["PID"].astype(str) + '_' + df_persons_NE["Area_MSOA"]

#Create a new column in 'df_persons_NE' for the unique ID --> PID_Area_MSOA
df_households_NE["HID_AreaOA"] = df_households_NE["HID"].astype(str) + '_' + df_households_NE["Area_OA"]


In [13]:
#Join df_households_NE with df_MSOA_OA_2011_NE_list in order to get MSOA level within 
#df_households_NE and then pass the Area_OA to df_persons_NE based on the MSOA level value
df_households_NE = df_households_NE.merge(df_MSOA_OA_2011_NE_list, left_on='Area_OA', right_on='oa', how='left')

In [14]:
#Rename the column names of 'df_persons_NE'
df_households_NE = df_households_NE.rename({'msoa': 'Area_MSOA'}, axis=1)

In [15]:
#Merge df_persons_NE with df_households_NE in order to get the Area_OA for each person
df_persons_NE = pd.merge(df_persons_NE, df_households_NE,  how='left', left_on=['HID','Area_MSOA'], right_on = ['HID','Area_MSOA'])

In [16]:
# Create a new column in df_households_NE combining HRPID with Area_MSOA in order to then match the Household refernce person (HRPID)
# from the household dataframe with the person in the df_persons_NE
df_households_NE["HRPID_AreaMSOA"] = df_households_NE["HRPID"].astype(str) + '_' + df_households_NE["Area_MSOA"]


In [17]:
# # Merge df_persons_NE with df_households_NE AGAIN in order to get the HRPID_Area_MSOA for each person
# The goal with this is that the new LC4605_C_NSSEC will be only assigned to the household reference person
# The rest of member of the household will have an empty value in this column

df_persons_NE = pd.merge(df_persons_NE, df_households_NE,  how='left', left_on=['PID_AreaMSOA'], right_on = ['HRPID_AreaMSOA'])

Keep only those necesary columns and remane 

In [20]:
#KEEP ONLY the required columns
df_persons_NE = df_persons_NE[['PID_AreaMSOA','PID','Area_MSOA_x', 'Area_OA_x', 'Sex','Age','Ethnic','HID_x', 'HID_AreaOA_x', 'LC4408_C_AHTHUK11_x', 'LC4404_C_SIZHUK11_x', 'LC4605_C_NSSEC_x', 'LC4202_C_CARSNO_x', 'LC4202_C_ETHHUK11_x', 'HRPID_AreaMSOA', 'LC4605_C_NSSEC_y', 'QS420_CELL_x']]

In [ ]:
#Rename the column names of 'df_persons_NE'
df_persons_NE = df_persons_NE.rename({'LC4605_C_NSSEC_y': 'NSSEC'}, axis=1)

Remove people that do not belong to any househodls and those that are assigned to households that are not residential (QS420_CELL = -2) or education (QS420_CELL = 26)

In [21]:
# Remove rows from df_persons_NE which HIP = (-1), as they are not assigned to any household
# They should be considered as leftovers in the process of matching persons with households

df_persons_NE = df_persons_NE[df_persons_NE['HID'] != -1]

2666722

In [25]:
## Remove those PEOPLE that do not belong to residential households (QS420_CELL = -2) or to student accommodations (QS420_CELL = 26)

residential_type_list = [-2,26]
df_persons_NE = df_persons_NE.loc[(df_persons_NE['QS420_CELL_x'].isin(residential_type_list))]

len(df_persons_NE)

2645517

Remove households that are assigned to households that are not residential (QS420_CELL = -2) or education (QS420_CELL = 26)

In [24]:
## Remove those HOUSEHOLDS that do not belong to residential households (QS420_CELL = -2) or to student accommodations (QS420_CELL = 26)

residential_type_list = [-2,26]
df_households_NE_extended = df_households_NE.loc[(df_households_NE['QS420_CELL'].isin(residential_type_list))]

len(df_households_NE_extended)

1217345

Now, both dataframes are cleaned and ready to be used

Export dataframes into csv files

In [26]:
df_persons_NE_clean = df_persons_NE

df_households_NE_clean = df_households_NE

In [27]:
# Export 'df_persons_NE_clean' and 'df_households_NE_clean' into *.csv files

df_persons_NE_clean.to_csv(r'C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\SPENSER\Data\NE_only\df_persons_NE_clean.csv', encoding='utf-8', header=True)
df_households_NE_clean.to_csv(r'C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\SPENSER\Data\NE_only\df_households_NE_clean.csv', encoding='utf-8', header=True)
